In [1]:
from src.exp_logger import logger

import pyterrier as pt  # type: ignore
import json
from src.load_index import setup_system

logger.setLevel("WARNING")

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
index, topics, qrels = setup_system("WT")


20:18:14.060 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 3,4 GiB of memory would be required.
Loaded index with  1570734 documents.


In [3]:
meta = index.getMetaIndex()

In [4]:
id_no = {}

while True:
    try:
        id_no[len(id_no)] = meta.getItem("docno", len(id_no))
    except:
        print("Done")
        break

Done


In [5]:
no_id = {v: k for k, v in id_no.items()}

In [6]:
def make_passages(doc):
    passages = doc.split("\n")

    # remove short passages
    passages = [passage for passage in passages if len(passage.strip().split(" "))>=3]

    # split into chuncs of 50 words
    result = []
    gathered_passages = ""

    for passage in passages:
        # print("passage:", len(passage.split(" ")), "gathered Passage:", len(gathered_passages.split(" ")))

        if len(gathered_passages.split(" ")) + len(passage.split(" ")) <= 65:
            gathered_passages = gathered_passages  + " " + passage
            # print("Gathering further")
        elif len(gathered_passages.split(" ")) + len(passage.split(" ")) > 65:
            if len(gathered_passages.split(" ")) == 0:
                result.append(passage.strip())
            else:
                result.append(gathered_passages.strip())
                # print(len(gathered_passages.split(" ")))

                gathered_passages = passage
    return result


def clean(row):
    doc = row["doc"]
    doc = doc.replace(row["docno"], "")
    result = make_passages(doc)
    return result

def find_max_list(list):
    list_len = [len(i) for i in list]
    return max(list_len)

In [16]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

try:
    with open("done.txt", "r") as fin:
        done = fin.read().splitlines()
except:
    done = []

from tqdm import tqdm

# get relevant docs
rels = topics.merge(qrels[qrels["label"]>=1], on="qid")


for _, query in tqdm(topics.iterrows(),total=len(topics)):
    qid = query["qid"]
    query_text = query["query"]

    if str(qid) in done:
        continue

    ##### Get relevant passages #####
    passages = []
    rel_docs = rels[rels["qid"] == qid]["docno"].tolist()  # get relevant docs
    if not rel_docs:
        logger.warning(f"Skipping {qid}, not relevant docs found")
        continue


    for docno in rel_docs:
        docid = no_id[docno]
        doc = index.getMetaIndex().getItem("text", docid)
        cleaned_doc = doc.replace(docno, " ").strip()
        doc_pass = make_passages(cleaned_doc)
        passages.append(doc_pass)
        logger.info(f"Found {len(doc_pass)} passages for {docno}")
    



    ##### Get not relevant but challenging passages #####

    # sample graded not rel but retrieved docs
    base = bm25(topics[topics["qid"]==qid])  # baseline

    if len(base) == 0:
        logger.warning(f"Skipping {qid}, no docs found")
        continue

    logger.info(f"Retrieved {len(base)} docs for {qid}")

    gradet = base.merge(qrels, on=["qid", "docno"])  # add grading    
    gradet = gradet[gradet["label"]==0]["docno"].tolist()  # get not rel docs
    logger.info(f"Found {len(gradet)} graded and not relevant docs for {qid}")


    not_rel_passages = []
    for docno in gradet:
        docid = no_id[docno]
        doc = index.getMetaIndex().getItem("text", docid)
        cleaned_doc = doc.replace(docno, " ").replace("|", "").strip()
        doc_pass = make_passages(cleaned_doc)
        not_rel_passages.append(doc_pass)

    
    tot_passages = sum([len(i) for i in passages])
    tot_not_rel_passages = sum([len(i) for i in not_rel_passages])
    logger.info(f"Found {tot_passages} relevant passages and {tot_not_rel_passages} not relevant passages")
    if tot_passages > tot_not_rel_passages:
        print("Not enough not relevant passages")
        # sample from not gradet not rel but retrieved docs
        # additional
        not_rel_docs_all = base["docno"].to_list()
        set(not_rel_docs_all)-set(gradet)-set(rel_docs)

        i = 0
        while tot_not_rel_passages < tot_passages or i == len(not_rel_docs_all)-1 :
            docid = no_id[not_rel_docs_all[i]]
            doc = index.getMetaIndex().getItem("text", docid)
            cleaned_doc = doc.replace(docno, " ").replace("|", "").strip()
            doc_pass = make_passages(cleaned_doc)
            not_rel_passages.append(doc_pass)    
            i+=1
            tot_not_rel_passages += len(doc_pass)
            if i == len(not_rel_docs_all)-1:
                
                continue




    
    ##### Assamble triplets #####
    non_rel_passage_pool = []
    for i in range(0, find_max_list(not_rel_passages)):
        for list_d in not_rel_passages:
            try:
                item = list_d[i]
                if not item:
                    continue  # TODO fix empty non rel passages
                non_rel_passage_pool.append(item)
            except IndexError:
                pass
        
    
    triplets = []
    c = 0
    for rel_doc in passages:
        for rel_passage in rel_doc:
            non_rel_passage = non_rel_passage_pool[c]
            triplets.append([query_text, rel_passage, non_rel_passage])
            c+=1

    with open("passages.jsonl", "+a") as fin:
        for triplet in triplets:
            fin.write(json.dumps(triplet))
            fin.write("\n")

    with open("done.txt", "+a") as fin:
        fin.write(str(qid))
        fin.write("\n")
    
    print("Done with", qid)


  0%|          | 0/672 [00:00<?, ?it/s][2023-04-29 21:27:24][WARNING] src.exp_logger: Skipping q062216451, not relevant docs found
[2023-04-29 21:27:24][WARNING] src.exp_logger: Skipping q0622724, not relevant docs found
[2023-04-29 21:27:24][WARNING] src.exp_logger: Skipping q062222125, not relevant docs found
[2023-04-29 21:27:24][WARNING] src.exp_logger: Skipping q06223848, not relevant docs found
[2023-04-29 21:27:24][WARNING] src.exp_logger: Skipping q06223898, not relevant docs found
[2023-04-29 21:27:24][WARNING] src.exp_logger: Skipping q062218529, not relevant docs found
[2023-04-29 21:27:24][WARNING] src.exp_logger: Skipping q062219081, not relevant docs found
[2023-04-29 21:27:24][WARNING] src.exp_logger: Skipping q06228499, no docs found
[2023-04-29 21:27:24][WARNING] src.exp_logger: Skipping q06229033, not relevant docs found
[2023-04-29 21:27:24][WARNING] src.exp_logger: Skipping q062210081, not relevant docs found
[2023-04-29 21:27:24][WARNING] src.exp_logger: Skipping q

Not enough not relevant passages
Done with q062219519


 83%|████████▎ | 558/672 [00:00<00:00, 609.65it/s]

Done with q062219521
Done with q062219533
Done with q062219608
Done with q062219780
Not enough not relevant passages
Done with q062219826
Done with q062219889
Done with q062219914
Not enough not relevant passages
Done with q062219963
Done with q062219965
Not enough not relevant passages
Done with q062220046
Not enough not relevant passages
Done with q062220131
Not enough not relevant passages
Done with q062220278
Done with q062220336
Done with q062220412
Done with q062220442
Not enough not relevant passages
Done with q062220476
Done with q062220574
Not enough not relevant passages
Done with q062220605
Done with q062220619
Done with q062220753
Not enough not relevant passages
Done with q062220762
Done with q062220773


 86%|████████▋ | 580/672 [00:18<00:04, 22.77it/s] 

...

Done with q062225030


100%|█████████▉| 671/672 [01:27<00:00,  1.25it/s]

Done with q062225194


100%|██████████| 672/672 [01:28<00:00,  7.59it/s]

Not enough not relevant passages
Done with q062225197


In [14]:
not_rel_docs_all

['doc062209200074']